In [25]:
import json
import sqlite3
import mysql.connector
from experimentation.Sets import Datasets
from experimentation.Utils import TextColor


database = mysql.connector.connect(
  host="atenea.rmontanana.es",
  port=31428,
  user="stree",
  password="xtree",
  database="stree",
)
dt = Datasets(False, False, 'tanveer')

#
# CONFIGURABLE REPORT PARAMETERS
#
exclude_parameters = True
# classififer can be {any, stree, bagging, adaBoost, odte}
classifier = "stree"
title = "Best Hyperparameters found for datasets"

In [26]:
def find_best(dataset):
    cursor = database.cursor(buffered=True)
    if classifier == "any":
        command = f"select * from results r inner join reference e on r.dataset=e.dataset where r.dataset='{dataset}' "
    else:
        command = f"select * from results r inner join reference e on r.dataset=e.dataset where r.dataset='{dataset}' and classifier='{classifier}'" 
    command += " order by r.dataset, accuracy desc, classifier desc, type, date, time"
    cursor.execute(command)
    return cursor.fetchone()

In [27]:
lengths = (10, 8, 10, 10, 30, 3, 3, 9, 11)
if exclude_parameters:
    fields = ('Date', 'Time', 'Type', 'Classifier', 'Dataset', 'Nor', 'Std', 'Accuracy', 'Reference')
    lengths = (10, 8, 10, 10, 30, 3, 3, 9, 11)
else:
    fields = ('Date', 'Time', 'Type', 'Classifier', 'Dataset', 'Nor', 'Std', 'Accuracy', 'Reference', 'Parameters')
    lengths = (10, 8, 10, 10, 30, 3, 3, 9, 11, 30)
def report_header_content(title):
    length = sum(lengths) + len(lengths) - 1
    output = "\n" + "*" * length + "\n"
    title = title + f" -- {classifier} classifier --"
    num = (length - len(title) - 2) // 2
    num2 = length - len(title) - 2 - 2 * num
    output += "*" + " " * num + title + " " * (num + num2) + "*\n"
    output += "*" * length + "\n\n"
    lines = ""
    for item, data in enumerate(fields):
        output += f"{fields[item]:{lengths[item]}} "
        lines += "=" * lengths[item] + " "
    output += f"\n{lines}"
    return output
def report_header(exclude_params):
    print(
        TextColor.HEADER
        + report_header_content(title)
        + TextColor.ENDC
    )
def report_line(record, agg):
    accuracy = record[5]
    expected = record[10]
    if accuracy < expected: 
        agg['worse'] += 1
        sign = '-'
    elif accuracy > expected:
        agg['better'] += 1
        sign = '+'
    else:
        agg['equal'] +=1
        sign = '='
    output = f"{record[0]:%Y-%m-%d} {str(record[1]):>8s} {record[2]:10s} {record[3]:10s} {record[4]:30s}"
    output += f" {record[6]:3d} {record[7]:3d} {accuracy:8.7f} {expected:8.7f} {sign}"
    if not exclude_parameters:
        output += f" {record[8]}"
    return output
def report_footer(agg):
    print(
        TextColor.GREEN
        + f"we have better results {agg['better']:2d} times"
    )
    print(
        TextColor.RED
        + f"we have worse  results {agg['worse']:2d} times"
    )
    print(
        TextColor.MAGENTA
        + f"we have equal  results {agg['equal']:2d} times"
    )

In [28]:
report_header(title)
color = TextColor.LINE1
agg = {'equal': 0, 'better': 0, 'worse': 0}
for dataset in dt:
    record = find_best(dataset[0])
    if record is None:
        print(TextColor.FAIL + f"*No results found for {dataset[0]}")
    else:
        color = TextColor.MAGENTA if color == TextColor.LINE1 else TextColor.LINE1
        print(color + report_line(record, agg))
report_footer(agg)


******************************************************************************************************
*                   Best Hyperparameters found for datasets -- stree classifier --                   *
******************************************************************************************************

Date       Time     Type       Classifier Dataset                        Nor Std Accuracy  Reference   
========== ======== ========== ========== ============================== === === ========= =========== 
2020-11-13 12:04:24 crossval   stree      balance-scale                    1   0 0.9488000 0.9046280 +
2020-11-13 12:04:25 crossval   stree      balloons                         1   0 0.8666670 0.6625000 +
2020-11-13 12:04:25 crossval   stree      breast-cancer-wisc-diag          1   0 0.9789320 0.9743450 +
2020-11-13 12:04:25 crossval   stree      breast-cancer-wisc-prog          1   0 0.8284620 0.7993400 +
2020-11-13 12:04:25 crossval   stree      breast-cancer-wisc         

******************************************************************************************************
*                    Best Hyperparameters found for datasets -- any classifier --                    *
******************************************************************************************************

Date       Time     Type       Classifier Dataset                        Nor Std Accuracy  Reference   
========== ======== ========== ========== ============================== === === ========= =========== 

2020-11-13 12:04:24 crossval   stree      balance-scale                    1   0 0.9488000 0.9046280 +
2020-11-13 12:04:25 crossval   stree      balloons                         1   0 0.8666670 0.6625000 +
2020-11-16 13:34:47 gridsearch bagging    breast-cancer-wisc-diag          1   1 0.9806860 0.9743450 +
2020-11-11 23:10:52 gridsearch bagging    breast-cancer-wisc-prog          1   1 0.8337180 0.7993400 +
2020-11-16 13:34:49 gridsearch bagging    breast-cancer-wisc               1   1 0.9699790 0.9702560 -
2020-11-18 23:04:30 gridsearch odte       breast-cancer                    1   1 0.7449490 0.7382400 +
2020-11-16 13:37:13 gridsearch bagging    cardiotocography-10clases        1   1 0.7022580 0.8277610 -
2020-11-16 13:35:23 gridsearch bagging    cardiotocography-3clases         1   1 0.8490220 0.9201340 -
*No results found for conn-bench-sonar-mines-rocks
2020-11-13 12:04:27 crossval   stree      cylinder-bands                   1   0 0.6326670 0.7691410 -
2020-11-13 12:04:27 crossval   stree      dermatology                      1   0 0.9754540 0.9732780 +
2020-11-13 12:04:27 crossval   stree      echocardiogram                   1   0 0.8472930 0.8485270 -
2020-11-19 15:22:26 gridsearch bagging    fertility                        1   1 0.8900000 0.8840000 +
2020-11-13 12:04:28 crossval   stree      haberman-survival                1   0 0.7646750 0.7392540 +
2020-11-13 12:04:28 crossval   stree      heart-hungarian                  1   0 0.8299240 0.8204750 +
2020-11-13 12:04:28 crossval   stree      hepatitis                        1   0 0.8645160 0.8232030 +
2020-11-13 12:04:28 crossval   stree      ilpd-indian-liver                1   0 0.7426910 0.7150280 +
2020-11-16 13:34:59 gridsearch bagging    ionosphere                       1   1 0.9515490 0.9442150 +
2020-11-03 18:52:15 gridsearch odte       iris                             1   0 0.9933330 0.9786560 +
2020-11-13 12:15:39 crossval   stree      led-display                      1   0 0.7120000 0.7102000 +
*No results found for libras
*No results found for low-res-spect
2020-11-16 13:35:08 gridsearch bagging    lymphography                     1   1 0.8781610 0.8554050 +
2020-11-13 12:15:39 crossval   stree      mammographic                     1   0 0.8293720 0.8274720 +
2020-11-16 22:17:03 gridsearch bagging    molec-biol-promoter              1   1 0.9060610 0.8182690 +
*No results found for musk-1
2020-11-13 12:15:39 crossval   stree      oocytes_merluccius_nucleus_4d    1   0 0.8082210 0.8399630 -
2020-11-13 12:15:39 crossval   stree      oocytes_merluccius_states_2f     1   0 0.9119030 0.9299630 -
2020-11-13 12:15:40 crossval   stree      oocytes_trisopterus_nucleus_2f   1   0 0.7476910 0.8333330 -
2020-11-16 13:35:41 gridsearch bagging    oocytes_trisopterus_states_5b    1   1 0.8540800 0.9315790 -
2020-11-13 12:15:40 crossval   stree      parkinsons                       1   0 0.8461540 0.9202210 -
2020-11-13 12:15:41 crossval   stree      pima                             1   0 0.7800020 0.7671880 +
2020-11-13 12:15:41 crossval   stree      pittsburg-bridges-MATERIAL       1   0 0.8861470 0.8642860 +
2020-11-16 13:36:41 gridsearch bagging    pittsburg-bridges-REL-L          1   1 0.6766670 0.6959290 -
2020-11-16 13:22:50 gridsearch adaBoost   pittsburg-bridges-SPAN           1   1 0.7210530 0.6891300 +
2020-11-13 12:15:41 crossval   stree      pittsburg-bridges-T-OR-D         1   0 0.9023810 0.8743700 +
2020-11-13 12:15:41 crossval   stree      planning                         1   0 0.7255250 0.7255790 -
2020-11-13 12:15:41 crossval   stree      post-operative                   1   0 0.7222220 0.7117420 +
2020-11-13 12:15:41 crossval   stree      seeds                            1   0 0.9619050 0.9563030 +
2020-11-13 12:15:41 crossval   stree      statlog-australian-credit        1   0 0.6797100 0.6782810 +
2020-11-13 12:21:08 crossval   stree      statlog-german-credit            1   0 0.7620000 0.7562000 +
2020-11-16 13:36:12 gridsearch bagging    statlog-heart                    1   1 0.8518520 0.8422990 +
2020-11-16 14:04:13 gridsearch bagging    statlog-image                    1   1 0.9627710 0.9761940 -
2020-11-13 12:21:13 crossval   stree      statlog-vehicle                  1   0 0.8014130 0.8006730 +
*No results found for synthetic-control
2020-11-13 12:21:13 crossval   stree      tic-tac-toe                      1   0 0.9874350 0.9853850 +
2020-11-13 12:21:14 crossval   stree      vertebral-column-2clases         1   0 0.8290320 0.8491530 -
2020-06-26 11:03:03 gridsearch odte       wine                             0   0 0.9800000 0.9932810 -
2020-11-13 12:21:14 crossval   stree      zoo                              1   0 0.9704760 0.9603850 +
we have better results 29 times
we have worse  results 15 times
we have equal  results  0 times